In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn_crfsuite import metrics

import logging

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

# from ner_eval import collect_named_entities
# from ner_eval import compute_metrics
# from ner_eval import compute_precision_recall_wrapper

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [2]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [3]:
train_df

,sentence_id,words,labels
0,1380681275594305537,@ridwankamil,O
1,1380681275594305537,@KickAndyShow,O
2,1380681275594305537,@Metro_TV,O
3,1380681275594305537,Upaya,O
4,1380681275594305537,untuk,O
...,...,...,...
109513,1389452349618327554,^^,O
109514,1389452349618327554,��dm,O
109515,1389452349618327554,for,O
109516,1389452349618327554,order,O


In [4]:
eval_df

,sentence_id,words,labels
0,1380695624258002949,chairman,O
1,1380695624258002949,ke,O
2,1380695624258002949,laut,O
3,1380695624258002949,",",O
4,1380695624258002949,Secretary,O
...,...,...,...
27175,1389448392737132545,Demak,I-PER
27176,1389448392737132545,Dikukuhkan,O
27177,1389448392737132545,Gus,B-PER
27178,1389448392737132545,AMI,I-PER


In [5]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [6]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    # "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    # "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [7]:
model = NERModel('bert', 
                'outputs/checkpoint-594-epoch-1',
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )
# result, model_outputs, wrong_preds = model.eval_model(eval_df)
# result

In [18]:
model = NERModel('bert', 
                'indolem/indobert-base-uncased',
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

In [19]:
model.train_model(train_df,  eval_data=eval_df)

Epoch 1 of 1:   0%|          | 0/1 [00:00<?, ?it/s]d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\simpletransformers\ner\ner_model.py:739: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://py

(594,
 {'global_step': [594],
  'precision': [0.4381948986265533],
  'recall': [0.29619805481874445],
  'f1_score': [0.35346874175679244],
  'train_loss': [0.43477529287338257],
  'eval_loss': [0.3642743123357728]})

In [8]:
result, model_outputs, y_pred = model.eval_model(test_df)

Running Evaluation: 100%|██████████| 186/186 [00:29<00:00,  6.36it/s]


In [9]:
result

{'eval_loss': 0.3486134983919641,
 'precision': 0.49208025343189016,
 'recall': 0.32816901408450705,
 'f1_score': 0.39374735952682716}

In [17]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

In [11]:
df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [12]:
print(np.shape(y_test),np.shape(y_test))

(1485,) (1485,)


d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\numpy\core\fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [13]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 19.98it/s]


In [14]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'B-PROD'},
  {'SHOPEE': 'O'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'}]]

In [15]:
#Now we create the model again using the best estimators

print(metrics.flat_classification_report(
    y_test, y_pred, labels=label_list, digits=3
    ))

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA', 'B-EV', 'I-EV'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [33814, 33379]

In [1]:
for i in range(len(y_test)):
    if(len(y_test[i])!=len(y_pred[i])):
        print(x_test[i])
        print(len(x_test[i]))
        print(y_test[i])
        print(len(y_test[i]))
        print(y_pred[i])
        print(len(y_pred[i]))
        print()

NameError: name 'y_test' is not defined

In [ ]:
test_df.iloc[0]

sentence_id    1380685209075544064
words                  @GyuuPotter
labels                           O
Name: 0, dtype: object